In [183]:
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests

In [179]:
def scrape_player_links(pageHTML,linkList):
    soup = BeautifulSoup(pageHTML, 'lxml')
    playerTD = soup.findAll('td', {'class':'name selected'})
    for player in playerTD:
        link = player.find('a')['href'].encode('ascii','ignore').strip()
        linkList.append(link)
    return linkList

In [180]:
#  Use the instructions found here to install PhantomJS on Ubuntu: 
#       https://www.vultr.com/docs/how-to-install-phantomjs-on-ubuntu-16-04

# Open a PhantomJS web browser and direct it to the DEA's dropbox search page
browser = webdriver.PhantomJS()
browser.get('http://www.nfl.com/combine/tracker#day=fullresults')
browser.implicitly_wait(100)

In [181]:
#navButtonRow = soup.findAll('li', {'class':'page'})
#test = xpath_soup(navButtonRow[1])
#elm = browser.find_element_by_xpath(test)
#elm = browser.find_element_by_xpath('/html/body/div[4]/div[3]/div/div[2]/div[1]/div[2]/div[1]/div[2]/table/tfoot/tr/td/ol/li[4]')

# Storage container for all player links
urlList = []

# Extract the original page information
html = browser.page_source
urlList = scrape_player_links(html, urlList)
#urlList

In [182]:
# Extract the navigation bar at the bottom of the page for navigation
navBar = browser.find_elements_by_class_name('page')
pageCount = len(navBar)

# Advance to the next page (page 2)
buttons = browser.find_elements_by_class_name('page')
buttons[1].click()

# Advance to the next page and repeat the scraping process (for all remaining pages)
for page in range(2,pageCount):
    html = browser.page_source
    urlList = scrape_player_links(html,urlList)
    buttons = browser.find_elements_by_class_name('page')
    buttons[page].click()
    
# Grab the last page (since the loop won't iterate through the final page)
html = browser.page_source
urlList = scrape_player_links(html,urlList)

In [175]:
#urlList

Now that we have all the player links, we can access each individual player page to obtain detailed information.

In [206]:
playerInfoList = []
playerCount = 0

headers = {"User-agent":
           "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.80 Safari/537.36"}

for link in urlList:
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.content,'lxml')

    # Basic info is react_id = 153
    #basicInfo = soup.find('div', {'data-reactid':'153'})
    basicInfo = soup.findAll('script')[5].text
    basicInfo = basicInfo.split('__INITIAL_DATA__ = ')[1]
    basicInfo = basicInfo.split(';\n  ')[0]
    basicInfo = basicInfo.encode('ascii','ignore')
    basicInfo = basicInfo.replace('\\"','')
    
    temp = json.loads(basicInfo)
    
    player = {}
    
    player['picURL'] = temp['instance']['prospect']['headshot']['asset']['url']                  # pictureURL
    player['nameFirst'] = temp['instance']['prospect']['person']['firstName']                    # first name
    player['nameLast'] = temp['instance']['prospect']['person']['lastName']                      # last name
    player['college'] = temp['instance']['prospect']['currentCollege']                           # college
    player['conference'] = temp['instance']['prospect']['collegeConference']                     # conference
    player['position'] = temp['instance']['prospect']['position']                                # position
    player['homeCity'] = temp['instance']['prospect']['person']['hometown']                      # hometown-city
    player['homeState'] = temp['instance']['prospect']['homeState']                              # hometown-state
    player['schoolYear'] = temp['instance']['prospect']['collegeClass']                          # year of school
    player['height'] = temp['instance']['prospect']['height']                                    # height
    player['weight'] = temp['instance']['prospect']['weight']                                    # weight
    player['lengthArm'] = temp['instance']['prospect']['armLength']                              # arm length
    player['lengthHand'] = temp['instance']['prospect']['handSize']                              # hand length
    player['grade'] = temp['instance']['prospect']['grade']                                      # prospect grade
    player['combineID'] = temp['instance']['prospect']['combineData']['combineNumber']           # combine ID
    player['combine40'] = temp['instance']['prospect']['combineData']['fortyYardDashResult']     # 40 yard dash
    player['combineBench'] = temp['instance']['prospect']['combineData']['benchResult']          # bench press
    player['combineVert'] = temp['instance']['prospect']['combineData']['verticalJumpResult']    # vertical jump
    player['combineBroad'] = temp['instance']['prospect']['combineData']['broadJumpResult']      # broad jump
    player['combineCone'] = temp['instance']['prospect']['combineData']['threeConeDrillResult']  # 3 Cone Drill
    player['combine20'] = temp['instance']['prospect']['combineData']['twentyYardShuttleResult'] # 20 yard shuttle run
    player['combine60'] = temp['instance']['prospect']['combineData']['sixtyYardShuttleResult']  # 60 yard shuttle run   
    player['overview'] = temp['instance']['prospect']['overview']                                # player Overview
    player['sources'] = temp['instance']['prospect']['sourcesTellUs']                            # info from sources
    player['strengths'] = temp['instance']['prospect']['strengths']                              # player strengths
    player['weaknesses'] = temp['instance']['prospect']['weaknesses']                            # player weaknesses
    player['bottomLine'] = temp['instance']['prospect']['bottomLine']                            # player bottom-line
    player['nflComp'] = temp['instance']['prospect']['nflComparison']                            # NFL comparison
    player['projection'] = temp['instance']['prospect']['draftProjection']                       # projected draft status
    player['profileAuthor'] = temp['instance']['prospect']['profileAuthor']                      # Profile Author
    
    playerInfoList.append(player)
    playerCount+=1
    if (playerCount%25==0): print(playerCount)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
276
277
278
279
280
281
282
283
284
285
286
287
2

In [207]:
# Some values for hometown-City capture the state as well (let's fix this)
for player in playerInfoList:
    if (isinstance(player['homeCity'], basestring)):
        if ', ' in  player['homeCity']: 
            oldCity = player['homeCity']
            player['homeCity'] = oldCity.split(', ')[0]
            player['homeState'] = oldCity.split(', ')[1]

In [208]:
# Export the data set
filename = 'combine_2018.json'
with open(filename, 'wt') as out:
    json.dump(playerInfoList, out, sort_keys=True, indent=4, separators=(',', ': ')) 